# Data imputation

Purpose:
The goal of the Notebook is to handle any nulls in the columns before preprocessing

In [4]:
base_dataset = pd.read_csv('../data/train_base.csv')

In [13]:
#since we found one-hot encoding better than label encoding, we'll apply that first
# convert string cols to categorical
categorical_cols = [base_dataset.columns[i] for i, x in enumerate(base_dataset.dtypes) if x == 'object']
base_dataset[categorical_cols] = base_dataset[categorical_cols].astype('category')

## convert categorical to one hot encodings
base_dataset = pd.get_dummies(base_dataset, drop_first=True)
base_dataset.head(1)

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Policy Duration Mins,Gender_Male,Marital Status_Married,Marital Status_Single,Education Level_High School,Education Level_Master's,Education Level_PhD,Occupation_Self-Employed,Occupation_Unemployed,Location_Suburban,Location_Urban,Policy Type_Comprehensive,Policy Type_Premium,Customer Feedback_Good,Customer Feedback_Poor,Smoking Status_Yes,Exercise Frequency_Monthly,Exercise Frequency_Rarely,Exercise Frequency_Weekly,Property Type_Condo,Property Type_House
0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,2869.0,518866.882265,False,True,False,False,False,False,True,False,False,True,False,True,False,True,False,False,False,True,False,True


# Median based imputations

In [26]:
base_dataset_path = "../data/"
dataset_version = "03_"
y_col = "Premium Amount"

## 1. simple median imputation

In [20]:
simple_median_dataset = base_dataset.copy()

simple_median_dataset = simple_median_dataset.fillna(simple_median_dataset.median())

file_name = dataset_version+ "imputation_median.csv"
save_path = os.path.join(base_dataset_path,file_name)

simple_median_dataset.to_csv(save_path,index=False)


## 2. median imputation (after clipping)

For now we'll use quantile clipping instead of z-score since some colmns are skewed

In [28]:
clipped_median_dataset = base_dataset.copy()


# Apply clipping column-wise
for column in clipped_median_dataset.columns:
    if clipped_median_dataset[column].dtype in ['float64', 'int64'] and column != y_col:  # Only process numeric columns
        lower = clipped_median_dataset[column].quantile(0.01)
        upper = clipped_median_dataset[column].quantile(0.99)
        clipped_median_dataset[column] = clipped_median_dataset[column].clip(lower, upper)

file_name = dataset_version+ "imputation_clipped_median.csv"
save_path = os.path.join(base_dataset_path,file_name)

clipped_median_dataset.to_csv(save_path,index=False)

## 3. median imputation (cleared outliers + grouping-derived medians)